### Test Inference Yolo V4 Tiny using OpenCV DNN

- Run this notebook in your vscode

In [ ]:
import cv2 
import numpy as np
import utils 
utility = utils.Utils()

In [3]:
# define class names, remember in notebook 4.2 we are training yolo v4 tiny with single class `scissors`
classNames = ["scissors"]

- put the downloaded `.cfg` and `.weights` file from notebook 4.2 into `model/` directory,

In [ ]:
# load trained Yolo V4 tiny weight and config
model = "model/custom-yolov4-tiny-detector_last.weights"
config = "model/custom-yolov4-tiny-detector.cfg"
net = cv2.dnn.readNet(model, config)


# load sample image & convert to blob
img = cv2.imread("image2.jpg")
resize_h, resize_w = 320, 320 
blob = cv2.dnn.blobFromImage(img, 1/255.0, (resize_w, resize_h), (0, 0, 0), swapRB=True, crop=False)


# do forward pass (inferencing)
net.setInput(blob)
layerOutput = net.getUnconnectedOutLayersNames()
output = net.forward(layerOutput)


# do postprocessing 
img = utility.postprocess_darknet(output, img, classNames, confThreshold = 0.2, nmsThreshold = 0.3, font_size=0.5, 
                        color=(255,127,0), text_color=(255,255,255))


# show result 
cv2.imshow("detection result", img)
cv2.waitKey()
cv2.destroyAllWindows()

- Inference Yolo V4 Tiny From camera

In [7]:
# load trained Yolo V4 tiny weight and config
model = "model/custom-yolov4-tiny-detector_last.weights"
config = "model/custom-yolov4-tiny-detector.cfg"
net = cv2.dnn.readNet(model, config)

# load video
cap = cv2.VideoCapture(0)
           
# iterate for each frame in video
while cap.isOpened():
    
    # get image on each frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # convert to blob
    resize_h, resize_w = 320, 320 # use smaller image size to reduce computation 
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (resize_w, resize_h), (0, 0, 0), swapRB=True, crop=False)

    # do a net forward (inferencing)
    net.setInput(blob)
    layerOutput = net.getUnconnectedOutLayersNames()
    output = net.forward(layerOutput)

    # do postprocessing
    frame = utility.postprocess_darknet(output, frame, classNames, confThreshold = 0.1, nmsThreshold = 0.2, font_size=0.5, 
                        color=(255,127,0), text_color=(255,255,255))

    # show result
    cv2.imshow('Frame',frame)

    # wait 25ms per frame and close using 'q' 
    if cv2.waitKey(1) == ord('q'):
          break


cap.release()
cv2.destroyAllWindows()